# Imports

In [ ]:
from image_gen import ImageDataGenerator
from build_model import build_UNet
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import glob
import os
import h5py
from util import mkdirs
import numpy as np
import configparser
import ast
from keras.utils.np_utils import to_categorical
import sys

# Configuration

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')
section = 'training'

In [ ]:
valid_portion=config.getfloat(section, 'valid_portion')
batch_size = config.getint(section, 'batch_size')
classes = ast.literal_eval(config.get(section, 'classes') if config.has_option(section, 'classes') else config.get('general', 'classes'))
matrice_file = config.get(section, 'matrice_file') if config.has_option(section, 'matrice_file') else config.get('saving', 'output_file')
experiment_folder = config.get(section, 'experiment_folder') if config.has_option(section, 'experiment_folder') \
                    else config.get('general', 'experiment_folder')
patch_size = config.getint(section, 'patch_size') if config.has_option(section, 'patch_size') else config.getint('general', 'patch_size')
nb_classes = len(classes)

# Creating necessary folder

In [ ]:
# creating folders
mkdirs(experiment_folder + "model/", 0o777)
mkdirs(experiment_folder + "matrices/", 0o777)
mkdirs(experiment_folder + "graphs/", 0o777)

# Functions definition

In [ ]:
def getTimestamp():
    """
        Return the timestamp
    """
    import datetime
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# TODO : Improve this and check if folder exists
def save_training_history(info, history):
    """
        Save the history of the model
    """
    
    # list all data in history
    print(history.history.keys())
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.gcf().savefig(info + '/loss_history.' + getTimestamp() + '.jpg')
    # plt.show()

    # summarize history for dice_coef
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.gcf().savefig(info + '/acc_history.' + getTimestamp() + '.jpg')
    # plt.show()

    # history to json file
    import json
    with open(info + '/log.' + getTimestamp() + '.json', 'w') as fp:
        json.dump(history.history, fp, indent=True)

# Code execution

In [ ]:
# Get the data
h5f = h5py.File(matrice_file,'r')

# Create the sets for training and validation
X_train, y_train = np.array([]).reshape((0,patch_size,patch_size,3)), np.array([]).reshape((0,patch_size,patch_size,nb_classes))
X_val, y_val = np.array([]).reshape((0,patch_size,patch_size,3)), np.array([]).reshape((0,patch_size,patch_size,nb_classes))

# Add the data in the training and validation sets
for key, val in classes.items():
    imgs = h5f[key+'_imgs'][:]
    masks = h5f[key+'_masks'][:]
    n_train = int(imgs.shape[0] * (1-valid_portion))
    
    X_train = np.concatenate((X_train,imgs[:n_train]))
    y_train_cat=to_categorical(masks[:n_train], num_classes=nb_classes) #Converting mask to one-hot encoded vectors 
    y_train_cat=y_train_cat.reshape((masks[:n_train].shape[0],masks[:n_train].shape[1],masks[:n_train].shape[2],y_train_cat.shape[1]))
    y_train = np.concatenate((y_train,y_train_cat))
    
    X_val = np.concatenate((X_val,imgs[n_train:]))
    y_val_cat=to_categorical(masks[n_train:], num_classes=nb_classes) #Converting mask to one-hot encoded vectors 
    y_val_cat=y_val_cat.reshape((masks[n_train:].shape[0],masks[n_train:].shape[1],masks[n_train:].shape[2],y_val_cat.shape[1]))
    y_val = np.concatenate((y_val,y_val_cat))
h5f.close()

In [ ]:
#TODO : Find a proper way to create a custom loss/accuracy

from keras import backend as K
from keras.backend.common import _EPSILON
import tensorflow as tf

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
        x: An object to be converted (numpy array, list, tensors).
        dtype: The destination type.
    # Returns
        A tensor.
    """
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def binary_crossentropy2K(output, target, from_logits=False):
    """Binary crossentropy between an output tensor and a target tensor.
    # Arguments
        output: A tensor.
        target: A tensor with the same shape as `output`.
        from_logits: Whether `output` is expected to be a logits tensor.
            By default, we consider that `output`
            encodes a probability distribution.
    # Returns
        A tensor.
    """
    # Note: tf.nn.sigmoid_cross_entropy_with_logits
    # expects logits, Keras expects probabilities.
    if not from_logits:
        # transform back to logits
        epsilon = _to_tensor(_EPSILON, output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon, 1 - epsilon)
        output = tf.log(output / (1 - output))

    return tf.nn.sigmoid_cross_entropy_with_logits(labels=target,
                                                   logits=output)

def binary_crossentropy2(y_true, y_pred):
    #print(binary_crossentropy2K(y_pred, y_true))
    #print(K.mean(binary_crossentropy2K(y_pred, y_true), axis=-1))
    return binary_crossentropy2K(y_pred, y_true)

In [ ]:
# Generate the model
inp_shape = X_train.shape[1:]
UNet = build_UNet(inp_shape, nb_classes)
#UNet.compile(optimizer='adam', loss=softmax_sparse_crossentropy_ignoring_last_label, metrics=[sparse_accuracy_ignoring_last_label])
UNet.compile(optimizer='adam', loss=binary_crossentropy2, metrics=['binary_accuracy'])

plot_model(UNet, os.path.join(experiment_folder,"model","model.png"), show_shapes=True)

In [ ]:
# Data Augmentation
# TODO : Put the parameters in config
train_gen = ImageDataGenerator(nb_classes=3,
                               rotation_range=180,
                                width_shift_range=0.3,
                                height_shift_range=0.3,
                                rescale=1.,
                                zoom_range=0.5,
                                horizontal_flip = True,
                                vertical_flip = True,
                                fill_mode='reflect',
                                cval=0)

#train_gen = ImageDataGenerator(rescale=1.)
test_gen = ImageDataGenerator(nb_classes=3,
                              rescale=1.)

In [ ]:
#Saving summary into the model folder
orig_stdout = sys.stdout
f = open(os.path.join(experiment_folder,"model","summary.txt"), 'w')
sys.stdout = f
print(UNet.summary())
sys.stdout = orig_stdout
f.close()

In [ ]:
# TODO : Put the parameters in config
model_file_format = os.path.join(experiment_folder, "matrices", 'model.{epoch:03d}.hdf5') 

checkpointer = ModelCheckpoint(model_file_format, period=10)

history = UNet.fit_generator(train_gen.flow(X_train, y_train, batch_size),
                            steps_per_epoch=(X_train.shape[0] + batch_size - 1) // batch_size,
                            epochs=100,
                            callbacks=[checkpointer],
                            validation_data=test_gen.flow(X_val, y_val),
                            validation_steps=(X_val.shape[0] + batch_size - 1) // batch_size)
    
save_training_history(os.path.join(experiment_folder,"graphs"), history)